<a href="https://colab.research.google.com/github/Shulliikkk/Researchs/blob/main/%D0%93%D0%A1%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Схема выведения КА на ГСО:


1.   Выход на НОО высотой 400 км (1 и 2 ступень РН)
2.   Сообщение первого импульса и переход на ГПО (2 ступень РН)
3.   Сообщение второго импульса и переход на ГСО (3 ступень РН)



### Расчет оптимальной массы 3 ступени РН, необходимой для перехода КА с ГПО на ГСО

In [23]:
#=======БЛОК КОНСТАНТ=======
# Все величины, которые будут использоваться в дальнейшем, будут выражены в СИ, если не оговорено инного 
from numpy import sqrt, sin, cos, exp, pi, log
M = 5.9722E24 # масса Земли
R = 6371E3 # радиус Земли
G = 6.6743E-11 #гравитационная постоянная
mu = G * M # гравитационный параметр Земли

In [20]:
# высота и радиус НОО
h = 400e3 
r1 = R + h
# высота и радиус ГСО
H = 35786e3
r2 = R + H

r_ratio = r2/r1 #соотношение радиусов орбит 

v1 = sqrt(mu/r1) #орбитальная скорость на НОО
v_apo = sqrt(mu * (2/r2 - 2/(r1 + r2))) #скорость в апогее ГПО

dv1 = v1*(sqrt(2*r_ratio/(r_ratio + 1)) - 1) #приращение скорости, необходимое для перехода с НОО на ГПО (пригодится в дальнейшем)
dv2 = v1/sqrt(r_ratio) * (1 - sqrt(2/(r_ratio + 1))) #приращение скорости в апогее ГПО, необходимое для перехода на ГСО

print('v1 =', v1)
print('dv1 =', dv1)
print('dv2 =', dv2)
print('dv1 + dv2 =', dv1 + dv2)
print('v1 + dv1 + dv2 =', v1 + dv1 + dv2)

v1 = 7672.618886595345
dv1 = 2399.3584848363753
dv2 = 1457.2301295227762
dv1 + dv2 = 3856.5886143591515
v1 + dv1 + dv2 = 11529.207500954497


Зададим некоторые параметры 3 ступени РН


In [6]:
# масса ПН (КА)
m_s = 5 

# параметры двигателя (тяга выбрана 20 кН, что примерно равно тяге двигателя разгонного блока Фрегат)
u3 = 500 # скорость истечения газов из сопла оностительно ракеты
mu3 = 40 # расход топлива
m_eng = 75 # масса двигателя
m_c = 20 # масса конструкции:)

M3_empt = m_s + m_eng + m_c

По формуле Циолковского рассчитаем массу топлива, необходимую для данного маневра

$$1457 = u_3\ln{\frac{m_s + m_{eng} + m_c + m_f}{m_s + m_{eng} + m_c}} = 500\ln{\frac{100 + m_f}{100}}$$ 

$$m_f = 1743 \; kg$$

Уточним её

In [19]:
dt = 0.001
for m_f in range(1740, 1900, 1):
  # координаты апоцентра ГПО
  x = -r2
  y = 0
  # компоненты начначальной скорости
  vx = 0
  vy = -v_apo
  m = M3_empt + m_f
  m0 = m
  f = True # флаг, показывающий, хватило ли топлива 
  while sqrt(vx**2 + vy**2) < v_apo + dv2:
    r = sqrt(x**2 + y**2) # модуль радиус-вектора
    x += vx*dt
    y += vy*dt
    ax = -mu/r**2 * x/r - mu3*u3/m * y/r
    ay = -mu/r**2 * y/r + mu3*u3/m * x/r
    vx += ax*dt
    vy += ay*dt
    if m - mu3*dt > M3_empt:
      m -= mu3*dt
    else:
      f = False
      break
  if f:
    print(m0)
    break

1845


Получим $1845$ и округлим до $1850$, поскольку расчеты велись для идеального случая точного попадания на ГСО

Таким образом уточненная масса топлива равна $m_f = 1850 \; kg$, а оптимальная масса всей 3 ступени $M_{III} = 1950 \; kg$


### Ориентировочне определие весовых характеристик ступеней РН


Введем коэффициент $\gamma$, показывающий сооотношение между массами ступеней:
$$\gamma = \frac{M_{II}}{M_I}$$

Характеристическая скорость 1 ступени:

$$\Delta v_I = u_1\ln{\frac{M_I + M_{II} + M_{III}}{kM_I + M_{II} + M_{III}}} = u_1\ln{\frac{M_{III} + M_{I}(1 + \gamma)}{M_{III} + M_{I}(k + \gamma)}}$$

Характеристическая скорость 2 ступени:

$$\Delta v_{II} = u_2\ln{\frac{M_{II} + M_{III}}{kM_{II} + M_{III}}} = u_2\ln{\frac{M_{III} + \gamma M_{I}}{M_{III} + k \gamma M_{I}}}$$

Общая характеристическая скорость:

$$\Delta v = \Delta v_I + \Delta v_{II} = v_1 + \Delta v_1 = 7672.6 + 2399.4 = 10072 \; m/s$$

$$\Delta v = u_1\ln{\frac{M_{III} + M_{I}(1 + \gamma)}{M_{III} + M_{I}(k + \gamma)}} + u_2\ln{\frac{M_{III} + \gamma M_{I}}{M_{III} + k \gamma M_{I}}}$$

Добавим к характеристической скорости потери на преодоление сопротивления атмосферы и притяжения Земли $\approx 1500 \; m/s$

Получим:

$$11600 = 4000\ln{\frac{1950 + M_{I}(1 + \gamma)}{1950 + M_{I}(0.05 + \gamma)}} + 3500\ln{\frac{1950 + \gamma M_{I}}{1950 + 0.05 \gamma M_{I}}}$$

Последовательно переберем $\gamma$ от 0.01 до 0.99 с шагом 0.01 и проведем оптимизацию по критерию $M_I + M_{II} \to min$

Решать данное уравнение будем методом бисекции

In [28]:
k = 0.05
M3 = 1950
u1 = 4000 
u2 = 3500 
dv = 11600
def f(m1):
  return u1*log((M3 + m1*(1 + gamma))/(M3 + m1*(k + gamma))) + u2*log((M3 + gamma*m1)/(M3 + k*gamma*m1)) - dv
gamma_opt = 1
M_opt = float('inf')
for gamma in range(1, 100):
  gamma /= 100
  l = 0
  r = 200e3
  e = 1
  while r - l > 2 * e:
    mid = (r + l) / 2
    if f(l) * f(mid) < 0:
      r = mid
    else:
      l = mid
  if M_opt > mid + gamma*mid:
    gamma_opt = gamma
    M_opt = mid + gamma*mid
    M1_opt = mid
    M2_opt = mid*gamma
print(gamma_opt)
print('Масса 1 ступени:', M1_opt)
print('Масса 2 ступени:', M2_opt)
print('Общая масса:', M1_opt + M2_opt + M3)
  #print(gamma, 'm2 =', round((mid*gamma + M3)/1000, 2), 'm =', round((mid + mid*gamma + M3)/1000, 2), u1*log((m + mid*(1 + gamma))/(m + mid*(k + gamma))), u2*log((m + gamma*mid)/(m + k*gamma*mid)))

0.12
Масса 1 ступени: 56327.81982421875
Масса 2 ступени: 6759.33837890625
Общая масса: 65037.158203125


Таким образом:

Масса 1 ступени $M_I \approx 56 \; ton$

Масса 2 ступени $M_{II} \approx 6.8 \; ton$

Масса 3 ступени $M_{III} \approx 1.95 \; ton$

Общая масса $M \approx 64.75 \; ton$